In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 5.5 MB 61.2 MB/s 
     |████████████████████████████████| 1.3 MB 58.0 MB/s 
     |████████████████████████████████| 182 kB 71.7 MB/s 
     |████████████████████████████████| 7.6 MB 48.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=afdb75aadaaee29ff989947d69c60500c9aa092edf4378221bde11b8181b3fe0
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
from sentence_transformers.SentenceTransformer import SentenceTransformer
import pickle
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import pandas as pd
import csv

filename = 'finalized_model_newLargeDataSet.sav'
loaded_model = pickle.load(open(filename, 'rb'))
model_preTrained = SentenceTransformer('all-MiniLM-L6-v2')

df = pd.read_csv('trainingDataSet.csv', sep=",", encoding='unicode_escape')
X, y, z = df[['Sentence1','Sentence2']], df['Similarity Point'].astype('float32'), df['Similarity Type']

X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(X, y, z, test_size=0.30, random_state=42)
# Two lists of sentences
sentences1 = []
for item in X_test['Sentence1']:
    sentences1.append(item)

sentences2 = []
for item in X_test['Sentence2']:
    sentences2.append(item)

similarity_type = []
for item in z_test:
  similarity_type.append(item)

#Compute embedding for both lists
embeddings1 = loaded_model.encode(sentences1, convert_to_tensor=True)
embeddings2 = loaded_model.encode(sentences2, convert_to_tensor=True)

embeddings1_preTrained = model_preTrained.encode(sentences1, convert_to_tensor=True)
embeddings2_preTrained = model_preTrained.encode(sentences2, convert_to_tensor=True)


#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)
consine_scores_preTrained = util.cos_sim(embeddings1_preTrained, embeddings2_preTrained)

#Output the pairs with their score
# for i in range(len(sentences1)):
#     print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

file = open('scores.csv', 'w', newline='')

with file:
    #identifying headers
    header = ['Sentence1', 'Sentence2', 'Cosine Score', 'Cosine Score Pre-Trained', 'Similarity Type']
    writer = csv.DictWriter(file, fieldnames=header)

    #writing data row-wise into the csv file
    writer.writeheader()
    for i in range(len(sentences1)):
      writer.writerow({'Sentence1': sentences1[i],
                      'Sentence2': sentences2[i],
                      'Cosine Score': cosine_scores[i][i].item(),
                      'Cosine Score Pre-Trained': consine_scores_preTrained[i][i].item(),
                      'Similarity Type': similarity_type[i]
      })

In [ ]:
import pandas as pd

df = pd.read_csv('scores.csv', sep=",", encoding='unicode_escape')
X, y, z = df[['Sentence1','Sentence2']], df[['Cosine Score', 'Cosine Score Pre-Trained']], df['Similarity Type']

#The cosine score of our model should be greater than the cosine score of the SBERT itself for the Jira-jira data
j = 0
k = 0
for i in range(len(z)):
  if z[i] == 1:
    if y['Cosine Score'][i] > y['Cosine Score Pre-Trained'][i]:
      j += 1
    else:
      k += 1
  
print('j and k:', j, k)

# The cosine score of our model should be lower than the cosine score of the SBERT itself for the nonJira-nonJira data

a = 0
b = 0
for i in range(len(z)):
  if z[i] == 2:
    if y['Cosine Score'][i] < y['Cosine Score Pre-Trained'][i]:
      a += 1
    else:
      b += 1

print('a and b:', a, b)

j and k: 4884 1094
a and b: 4538 7516


In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 9.0 MB/s 
     |████████████████████████████████| 88 kB 9.0 MB/s 
     |████████████████████████████████| 5.2 MB 43.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 66.2 MB/s 
     |████████████████████████████████| 1.1 MB 65.9 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=2340770 sha256=02bb8c2d5232a436786a49cf4ed1a4e204c89713e7bf487359523cd5eea426aa
  Stored in directory: /root/.cache/pip/wheels/93/78/2e/03ee191669a772e9653260aa3bd53e0b1a768751a9676e8c82
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=9ad5f020854fa11dc0ba5efd19785e0ac2a5ecb11598842eecca175a43005af5
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1f

In [ ]:
# This part of code is for running the segmentation code for our trained model to see how it works!
import pickle

import nltk
import numpy as np
import pickle
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words("english")

with open("Transcript2.txt", 'r') as f:
    corpus = f.readlines()
    corpus = [v.rstrip() for v in corpus if v != '\n']

# corpus = []
# for text in corpus0:
#     sentences = nltk.sent_tokenize(text)
#     text_arr = []
#     for sentence in sentences:
#         tokens = [w.lower() for w in nltk.word_tokenize(sentence)]
#         tokens = [w for w in tokens if w not in stopwords and len(w) > 1]
#         text_arr.append(' '.join(tokens))
#     corpus.append('.'.join(text_arr))

#docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))['data']
#docs = docs[0:10]

#loading our own trained model: (next 2 lines)
filename = 'finalized_model_newLargeDataSet.sav'
model = pickle.load(open(filename, 'rb'))
topic_model = BERTopic(min_topic_size=3, embedding_model=model)
#loading pre-trained model:
# topic_model = BERTopic(min_topic_size=3)
topics, probs = topic_model.fit_transform(corpus)

to_save = []
print("topics = {}, probs = {}".format(topics, probs))
for index in topic_model.get_topics():
    if index == -1:
        continue
    topic = topic_model.get_topic(index)
    repr = topic_model.get_representative_docs(index)
    topic_name = topic_model.get_topic_info(index)["Name"].tolist()[0]
    print("topic = {}, repr = {}".format(topic_name, repr))
    to_save.append({
        "topic_name": topic_name,
        "repr": repr
    })
np.save("segments.npy", np.array(to_save))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


topics = [-1, 0, 0, -1, -1, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, -1, 0, 0, 2, 0, 2, 0, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, -1], probs = [0.         1.         0.70466677 0.         0.         0.94577382
 0.82197801 1.         1.         1.         0.93744215 1.
 1.         0.88607384 1.         0.88663893 0.81027    0.
 1.         1.         1.         1.         1.         1.
 0.98257368 0.74854937 1.         0.76724651 0.74745515 0.88685424
 1.         1.         0.89289378 1.         0.         0.80433218
 0.92821822 0.         0.         0.92821822 0.        ]
topic = 0_the_to_of_like, repr = ['Also, can the Admin upload resources for a specific visit? Like just for the visit 2', 'You mean you want the nurses to be able to upload resources?', 'The format of the confirmation email after scheduling a meeting should be changed']
topic = 1_we_your_dm_with, repr = ["Nice try. That wasn't the issue though. It wouldn't even allow us to scheduled a trip. How about you provide

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#testing the topic segmentation of the trained model with the test data sentences
from sklearn.model_selection import train_test_split
import pandas as pd

import pickle

import nltk
import numpy as np
import pickle
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

df = pd.read_csv('trainingDataSet.csv', sep=",", encoding='unicode_escape')
X, y, z = df[['Sentence1','Sentence2']], df['Similarity Point'].astype('float32'), df['Similarity Type']

X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(X, y, z, test_size=0.30, random_state=42)

nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words("english")

for f in X_test['Sentence1']:
    corpus.append(f)
    corpus = [v.rstrip() for v in corpus if v != '\n']

# corpus = []
# for text in corpus0:
#     sentences = nltk.sent_tokenize(text)
#     text_arr = []
#     for sentence in sentences:
#         tokens = [w.lower() for w in nltk.word_tokenize(sentence)]
#         tokens = [w for w in tokens if w not in stopwords and len(w) > 1]
#         text_arr.append(' '.join(tokens))
#     corpus.append('.'.join(text_arr))

#docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))['data']
#docs = docs[0:10]

#loading our own trained model: (next 2 lines)
filename = 'finalized_model_newLargeDataSet.sav'
model = pickle.load(open(filename, 'rb'))
topic_model = BERTopic(min_topic_size=3, embedding_model=model)
#loading pre-trained model:
# topic_model = BERTopic(min_topic_size=3)
topics, probs = topic_model.fit_transform(corpus)

to_save = []
print("topics = {}, probs = {}".format(topics, probs))
for index in topic_model.get_topics():
    if index == -1:
        continue
    topic = topic_model.get_topic(index)
    repr = topic_model.get_representative_docs(index)
    topic_name = topic_model.get_topic_info(index)["Name"].tolist()[0]
    print("topic = {}, repr = {}".format(topic_name, repr))
    to_save.append({
        "topic_name": topic_name,
        "repr": repr
    })
np.save("segments.npy", np.array(to_save))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


topics = [821, 442, 518, 849, 478, -1, 267, 1099, 724, 465, 734, -1, 257, -1, 1028, 850, 811, -1, 610, 355, 739, 1167, 948, 358, 423, 745, 1145, 1214, 9, 4, 765, 336, 1046, 1194, 52, 253, 583, 828, 300, 470, -1, 569, 1124, 1012, 424, 239, 0, 315, -1, 1285, 669, 0, 133, 954, 485, 119, 365, 1338, 417, 1158, 223, 213, 93, 308, 146, 514, 5, 970, 135, 321, 37, 1375, 8, 5, 129, 634, 386, 874, 147, 173, 155, 147, 594, 52, 330, 233, 580, 341, -1, -1, -1, 502, 320, 444, 1011, 42, 287, 0, 922, 741, 1071, 646, 0, 341, 727, 42, 1170, 359, 1058, 1090, 265, 829, 89, 689, 111, 842, 51, 51, 1318, 1118, 1284, 1049, 1034, 345, 817, 756, 798, 80, 1312, 516, 295, 897, 17, -1, 726, 979, 213, 243, 16, 738, 5, 92, 0, 1110, 641, 9, 63, 399, 670, 468, 1108, 568, 563, 560, -1, 926, 646, 82, 158, 323, 439, -1, 2, 773, 981, 926, 35, 830, 682, 450, 208, 260, 710, 119, 286, 0, 453, 1150, 974, 579, 349, -1, 345, 64, 152, 261, 448, 100, 570, 422, 450, 553, 1202, 495, 24, 239, 189, 345, 690, 207, 124, 714, 1075, 847, 

In [ ]:
#testing the topic segmentation of the pre-trained model
from sklearn.model_selection import train_test_split
import pandas as pd

import pickle

import nltk
import numpy as np
import pickle
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

df = pd.read_csv('trainingDataSet.csv', sep=",", encoding='unicode_escape')
X, y, z = df[['Sentence1','Sentence2']], df['Similarity Point'].astype('float32'), df['Similarity Type']

X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(X, y, z, test_size=0.30, random_state=42)

nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words("english")

for f in X_test['Sentence1']:
    corpus.append(f)
    corpus = [v.rstrip() for v in corpus if v != '\n']

# corpus = []
# for text in corpus0:
#     sentences = nltk.sent_tokenize(text)
#     text_arr = []
#     for sentence in sentences:
#         tokens = [w.lower() for w in nltk.word_tokenize(sentence)]
#         tokens = [w for w in tokens if w not in stopwords and len(w) > 1]
#         text_arr.append(' '.join(tokens))
#     corpus.append('.'.join(text_arr))

#docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))['data']
#docs = docs[0:10]

#loading our own trained model: (next 2 lines)
# filename = 'finalized_model_newLargeDataSet.sav'
# model = pickle.load(open(filename, 'rb'))
# topic_model = BERTopic(min_topic_size=3, embedding_model=model)
#loading pre-trained model:
topic_model = BERTopic(min_topic_size=3)
topics, probs = topic_model.fit_transform(corpus)

to_save = []
print("topics = {}, probs = {}".format(topics, probs))
for index in topic_model.get_topics():
    if index == -1:
        continue
    topic = topic_model.get_topic(index)
    repr = topic_model.get_representative_docs(index)
    topic_name = topic_model.get_topic_info(index)["Name"].tolist()[0]
    print("topic = {}, repr = {}".format(topic_name, repr))
    to_save.append({
        "topic_name": topic_name,
        "repr": repr
    })
np.save("segments.npy", np.array(to_save))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

topics = [600, 314, -1, 904, 64, 1, 906, 1419, 172, 1344, 429, 430, 429, 905, -1, 429, 1424, 811, 216, 545, 727, 725, 202, 216, 667, 1178, 1181, 1179, 3, 1120, 726, 668, 1119, 1381, 34, 230, 546, 173, 1121, 284, 228, 505, 1011, 1289, 1092, 637, 0, 1297, 900, 1040, 621, 0, 107, 971, 597, 91, 287, 279, 970, 950, 200, 1308, 206, 281, 125, 485, 2, 935, -1, 390, 21, 903, 1041, 1503, 293, 571, 318, 898, 126, 607, 1132, 126, 813, 34, 276, 184, 360, 310, 239, 539, 776, 467, 260, 387, 999, 24, 765, 0, 785, 680, 514, 561, 0, 310, 724, 24, 1244, 311, 1029, 1049, 225, 652, 109, 397, 712, 979, 33, 33, 1232, 901, 1256, 1100, 1109, 297, 760, 718, 492, 54, 829, 434, 254, 844, 7, 599, 672, 635, 1332, 217, 4, 687, 2, 55, 0, 1217, 1090, 3, 42, 1047, 602, 305, 432, 516, 507, 499, 683, 976, 561, 317, 121, 273, 366, 551, -1, 745, 1020, 976, 63, 781, 1089, 372, 166, 220, 628, 91, 234, 0, 402, 1331, 945, 537, 288, 1451, 297, 35, 1407, 221, 374, 70, 536, 371, 372, 494, 1016, 486, 1470, 1235, 1340, 297, 642, 15